In [2]:
%load_ext Cython

In [53]:
%%cython
# cython: profile=True
import pstats, cProfile

from game import Game
from config import Config
import chess
import numpy as np
import gameimage.c as gic
from actionspace.maps import map_w, map_b
#from math import exp
cimport numpy as np
from libc.math cimport log, exp
np.import_array()


class Node:
    def __init__(self):
        # Values for tree representation
        self.parent = None
        self.children = {}
        self.image = None

        # Values for MCTS
        self.N = 0
        self.W = 0
        self.P = 0
        self.Q = 0

    def is_leaf(self):
        return len(self.children) == 0
    
    def reset(self):
        self.parent = None
        self.children = {}
        self.N = 0
        self.W = 0
        self.P = 0
        self.Q = 0


def run_mcts(config: Config, game: Game, network, root: Node = None, reuse_tree: bool = False, num_simulations: int = 100, num_sampling_moves: int = 30, add_noise: bool = True, pb_c_factor: float = 2.0):
    """Run Monte Carlo Tree Search algorithm from the current game state.

    Args:
        game (Game): Current game state.
        sim_num (int, optional): Number of simulations to run. Defaults to 100.

    Returns:
        Move, Root: Return the move to play and the root node with an updated tree layout.
    """
    rng = np.random.default_rng()
    pb_c_base = config.pb_c_base
    pb_c_init = config.pb_c_init

    # Initialize the root node
    # Allows us to reuse the image of the previous root node but also allows us to reuse the tree or make clean start
    if root is None:
        root = Node()
        root.N = 1
        _, policy_logits = make_predictions(root, game, network)
        expand(root, game, policy_logits)
    else:
        if reuse_tree:
            num_simulations = num_simulations - root.N + 1
        else:
            root.reset()
            root.N = 1
            _, policy_logits = make_predictions(root, game, network)
            expand(root, game, policy_logits)

    # Exploration noise used for full searches
    if add_noise:
        add_exploration_noise(config, root, rng)

    # Run the simulations
    for _ in range(num_simulations):
        node = root
        tmp_game = game.clone()

        # Select the best leaf node
        while node.is_leaf() is False:
            #move, node = select_leaf(config, node, pb_c_factor)  # Select best with ucb
            move, node = select_leaf(node, pb_c_base, pb_c_init, pb_c_factor)  # Select best with ucb
            tmp_game.make_move(move)

        # Evaluate the leaf node
        value, is_terminal = evaluate(tmp_game)

        # Expand if possible
        if is_terminal is False:
            value, policy_logits = make_predictions(node, tmp_game, network)
            expand(node, tmp_game, policy_logits)
        
        # Backpropagate the value
        update(node, -value)
    # return best_move, root
    temp = config.temp if game.history_len < num_sampling_moves else 0
    return select_move(root, rng, temp), root

def get_image(node: Node, game: Game):
    if node.parent is None: # Root node
        if node.image is None:
            node.image = gic.board_to_image(game.board)
        return node.image
    else:
        if node.parent.image is None:
            node.image = gic.board_to_image(game.board)
        else:
            node.image = gic.update_image(game.board, node.parent.image)
        return node.image

def make_predictions(node: Node, game: Game, network):
    image = get_image(node, game)
    value, policy_logits = network(image.astype(np.float32))
    return value, policy_logits

cdef expand(object node, object game, double[:] policy_logits):
    cdef dict policy = {}
    cdef bint to_play = game.to_play()
    cdef object move
    cdef str move_uci
    cdef int action
    cdef double p
    cdef double policy_sum = 0

    for move in game.board.legal_moves: # Acces via generator for speedup
        move_uci = chess.Move.uci(move)
        action = map_w[move_uci] if to_play else map_b[move_uci]
        p = exp(policy_logits[action])
        policy[move_uci] = p #exp(policy_logits[action])
        policy_sum += p

    for move_uci, p in policy.items():
        child = Node()
        child.parent = node
        child.P = p / policy_sum
        node.children[move_uci] = child

def evaluate(game: Game):
    if game.terminal():
        return game.terminal_value(game.to_play()), True
    else:
        return 0, False
    
def update(node: Node, value: float):
    node.N += 1
    node.W += value
    node.Q = node.W / node.N
    if not node.parent.parent is None:
        update(node.parent, -value)

cdef select_leaf(object node, float pb_c_base, float pb_c_init, float pb_c_factor):
    cdef str move
    cdef str bestmove
    cdef float ucb
    cdef float bestucb = -9999
    cdef object child
    cdef object bestchild

    for move, child in node.children.items():
        ucb = UCB(child.Q, child.P, child.N, node.N, pb_c_base, pb_c_init, pb_c_factor)
        if ucb > bestucb:
            bestucb = ucb
            bestmove = move
            bestchild = child
    
    return bestmove, bestchild 

cdef UCB(float cQ, float cP, int cN, int pN, float pb_c_base, float pb_c_init, float pb_c_factor):
    cdef float cpuct

    cpuct = (
        log((pN + pb_c_base + 1) / pb_c_base) + pb_c_init
    ) * pb_c_factor
    return cQ + cpuct * cP * pN**0.5 / (cN + 1)    

def select_move(node: Node, rng: np.random.Generator, temp = 0):
    moves = [move for move in node.children.keys()]
    visit_counts = [child.N for child in node.children.values()]
    if temp == 0:
        # Greedy selection (select the move with the highest visit count) 
        # If more moves have the same visit count, choose one randomly
        return moves[rng.choice(np.flatnonzero(visit_counts == np.max(visit_counts)))]
    else:
        # Use the visit counts as a probability distribution to select a move
        pi = np.array(visit_counts) ** (1 / temp)
        pi /= np.sum(pi)
        return moves[np.where(rng.multinomial(1, pi) == 1)[0][0]]

def add_exploration_noise(config: Config, node: Node, rng: np.random.Generator):
    noise = rng.gamma(config.root_dirichlet_alpha, 1, len(node.children))
    frac = config.root_exploration_fraction
    for i, (move, _) in enumerate(node.children.items()):
        node.children[move].P = node.children[move].P * (1 - frac) + noise[i] * frac

def fake_network(images):
    value = np.random.rand()
    policy = np.random.rand(4672)
    return value, policy

def test():
    config = Config()
    game = Game(config)
    for i in range(10):
        move = np.random.choice(game.legal_moves())
        game.make_move(move)
    for i in range(10):
        move, root = run_mcts(
            config,
            game,
            fake_network,
            None,
            False,
            500,
            30,
        )
        game.make_move(move)

cProfile.runctx("test()", globals(), locals(), "Profile.prof")

s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

Content of stderr:
In file included from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /home/tomaz/.cache/ipython/cython/_cython_magic_9fcbcb22ec76dd004bb665c059785249af43b7fc.c:1260:
/home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~Tue Feb 13 09:02:57 2024    Profile.prof

         12990671 function calls (12962293 primitive calls) in 6.930 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lin

In [56]:
%%cython
# cython: profile=True
import pstats, cProfile

from game import Game
from config import Config
import chess
import numpy as np
import gameimage.c as gic
from actionspace.maps import map_w, map_b
from math import exp, log
cimport numpy as np

import numpy as np
from actionspace.maps import map_w, map_b
from game import Game
from gameimage import c as gic
from model import predict_fn
from config import Config
from math import log, exp
import chess
import numba

# Each state-action pair (s, a) stores a set of statistics:
#     - N(s, a): visit count
#     - W(s, a): total action value
#     - Q(s, a): mean action value
#     - P(s, a): prior probability of selecting action a in state s

# A node stores the state. The action is the move that led to the state.
# For the root node the action is None. For all other nodes the action is the key in the children dictionary of the parent node.

class Node:
    def __init__(self):
        # Values for tree representation
        self.parent = None
        self.children = {}
        self.image = None

        # Values for MCTS
        self.N = 0
        self.W = 0
        self.P = 0
        self.Q = 0

    def is_leaf(self):
        return len(self.children) == 0
    
    def reset(self):
        self.parent = None
        self.children = {}
        self.N = 0
        self.W = 0
        self.P = 0
        self.Q = 0


def run_mcts(config: Config, game: Game, network, root: Node = None, reuse_tree: bool = False, num_simulations: int = 100, num_sampling_moves: int = 30, add_noise: bool = True, cpuct: float = None):
    """Run Monte Carlo Tree Search algorithm from the current game state.

    Args:
        game (Game): Current game state.
        sim_num (int, optional): Number of simulations to run. Defaults to 100.

    Returns:
        Move, Root: Return the move to play and the root node with an updated tree layout.
    """
    rng = np.random.default_rng()

    # Initialize the root node
    # Allows us to reuse the image of the previous root node but also allows us to reuse the tree or make clean start
    if root is None:
        root = Node()
        root.N = 1
        expand(root, game, network)
    else:
        if reuse_tree:
            num_simulations = num_simulations - root.N + 1
        else:
            root.reset()
            root.N = 1
            expand(root, game, network)

    # Exploration noise used for full searches
    if add_noise:
        add_exploration_noise(config, root, rng)

    # Run the simulations
    for _ in range(num_simulations):
        node = root
        tmp_game = game.clone()

        # Select the best leaf node
        while node.is_leaf() is False:
            move, node = select_leaf(config, node, cpuct)  # Select best with ucb
            tmp_game.make_move(move)

        # Evaluate the leaf node
        value, is_terminal = evaluate(tmp_game)

        # Expand if possible
        if is_terminal is False:
            value = expand(node, tmp_game, network)
        
        # Backpropagate the value
        update(node, -value)
    # return best_move, root
    temp = config.temp if game.history_len < num_sampling_moves else 0
    return select_move(root, rng, temp), root

def get_image(game: Game, node: Node):
    if node.parent is None: # Root node
        if node.image is None:
            node.image = gic.board_to_image(game.board)
        return node.image
    else:
        if node.parent.image is None:
            node.image = gic.board_to_image(game.board)
        else:
            node.image = gic.update_image(game.board, node.parent.image)
        return node.image

def expand(node: Node, game: Game, network=None):
    """Evaluate the given node using the neural network.

    Args:
        node (Node): Node to evaluate.z
    """

    image = get_image(game, node)
    # Get the policy and value from the neural network
    #value, policy_logits = predict_fn(network, image.astype(np.float32))
    #value = np.array(value)
    #policy_logits = np.array(policy_logits)
    value, policy_logits = network(image.astype(np.float32))

    policy = {}
    to_play = game.to_play()
    for move in game.board.legal_moves: # Acces via generator for speedup
        move_uci = chess.Move.uci(move)
        action = map_w[move_uci] if to_play else map_b[move_uci]
        policy[move_uci] = exp(policy_logits[action])

    policy_sum = sum(list(policy.values()))
    for move, p in policy.items():
        child = Node()
        child.parent = node
        child.P = p / policy_sum
        node.children[move] = child

    return value

def evaluate(game: Game):
    if game.terminal():
        return game.terminal_value(game.to_play()), True
    else:
        return 0, False
    
def update(node: Node, value: float):
    node.N += 1
    node.W += value
    node.Q = node.W / node.N
    if not node.parent.parent is None:
        update(node.parent, -value)

def select_leaf(config: Config, node: Node, cpuct = None):
    _, move, child = max(
        (ucb(config, node, child, cpuct), move, child)
        for move, child in node.children.items()
    )
    return move, child

def ucb(config: Config, parent: Node, child: Node, cpuct = None):
    # ucb = Q(s, a) + U(s, a)
    # U(s, a) = C(s)*P(s, a)*sqrt(N(s))/(1 + N(s, a))
        # Cs -> exploration constant
        # Psa -> child prior value
        # Ns -> parent visits count
        # Nsa -> child visits count
    # Q(s, a)
        # Q of child
    if cpuct is None: # AlphaZero exploration value, 1 for no exploration
        cpuct = (
            log((parent.N + config.pb_c_base + 1) / config.pb_c_base)
            + config.pb_c_init
        )
    return child.Q + cpuct * child.P * parent.N**0.5 / (child.N + 1)

def select_move(node: Node, rng: np.random.Generator, temp = 0):
    moves = [move for move in node.children.keys()]
    visit_counts = [child.N for child in node.children.values()]
    if temp == 0:
        # Greedy selection (select the move with the highest visit count) 
        # If more moves have the same visit count, choose one randomly
        return moves[rng.choice(np.flatnonzero(visit_counts == np.max(visit_counts)))]
    else:
        # Use the visit counts as a probability distribution to select a move
        pi = np.array(visit_counts) ** (1 / temp)
        pi /= np.sum(pi)
        return moves[np.where(rng.multinomial(1, pi) == 1)[0][0]]


def add_exploration_noise(config: Config, node: Node, rng: np.random.Generator):
    noise = rng.gamma(config.root_dirichlet_alpha, 1, len(node.children))
    frac = config.root_exploration_fraction
    for i, (move, _) in enumerate(node.children.items()):
        node.children[move].P = node.children[move].P * (1 - frac) + noise[i] * frac

def fake_network(images):
    value = np.random.rand()
    policy = np.random.rand(4672)
    return value, policy

def test():
    config = Config()
    game = Game(config)
    for i in range(11):
        move = np.random.choice(game.legal_moves())
        game.make_move(move)
    for i in range(10):
        move, root = run_mcts(
            config,
            game,
            fake_network,
            None,
            False,
            500,
            30,
        )
        game.make_move(move)

cProfile.runctx("test()", globals(), locals(), "Profile.prof")

s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

Content of stderr:
In file included from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /home/tomaz/.cache/ipython/cython/_cython_magic_dc702dd969fb30efa55dfd1ab478370dce4e9076.c:1262:
/home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~Tue Feb 13 09:07:11 2024    Profile.prof

         14174192 function calls (14145087 primitive calls) in 7.413 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lin

In [114]:
%%cython
# cython: profile=True
import pstats, cProfile

import chess
import numpy as np
cimport numpy as np
cimport cython
np.import_array()

DTYPE = np.intc
ctypedef int DTYPE_t

@cython.boundscheck(False)
@cython.wraparound(False)
cdef parse_piece_map(dict pieces_map):
    cdef int N = 8
    cdef Py_ssize_t row, col
    cdef int idx
    cdef np.ndarray board_np = np.zeros([N, N], dtype=DTYPE)
    cdef int[:, :] board_np_view = board_np
    cdef dict pieces = {
        "p": -1, "n": -2, "b": -3, "r": -4, "q": -5, "k": -6,
        "P": 1, "N": 2, "B": 3, "R": 4, "Q": 5, "K": 6
    }

    for idx, piece in pieces_map.items():
        row = idx // 8
        col = idx % 8
        board_np_view[row, col] = pieces[piece.symbol()]
    return board_np


@cython.boundscheck(False)
@cython.wraparound(False)
cdef parse_epd(epd):
    cdef int N = 8
    cdef np.ndarray board_np = np.zeros([N, N], dtype=DTYPE)
    cdef int space_between
    cdef Py_ssize_t i, j

    pieces = {
        "p": -1, "n": -2, "b": -3, "r": -4, "q": -5, "k": -6,
        "P": 1, "N": 2, "B": 3, "R": 4, "Q": 5, "K": 6
    }

    epd = epd.split(" ")
    piece_placement_rows = epd[0].split("/")

    for i, row in enumerate(piece_placement_rows):
        space_between = 0
        for j, col in enumerate(row):
            if col.isdigit():
                space_between += int(col) - 1
            else:
                board_np[i, j + space_between] = pieces[col]

    return board_np

""" @cython.boundscheck(False)
@cython.wraparound(False)
cdef pieces_onehot(np.ndarray[DTYPE_t, ndim=2] board_np, bint to_play):
    cdef int N = 8
    cdef int P = 6 # Number of pieces piece
    cdef np.ndarray onehot_w = np.zeros([N, N, P], dtype=DTYPE)
    cdef np.ndarray onehot_b = np.zeros([N, N, P], dtype=DTYPE)
    cdef Py_ssize_t i
    cdef Py_ssize_t _P = 6

    for i in range(_P):
        onehot_w[:, :, i] = board_np == i + 1
        onehot_b[:, :, i] = board_np == -(i + 1)
    if to_play: # If player one is white
        return onehot_w, onehot_b
    else: # If player one is black -> flip perspective
        return np.flip(np.flip(onehot_b, axis=0), axis=1), np.flip(np.flip(onehot_w, axis=0), axis=1) """

@cython.boundscheck(False)
@cython.wraparound(False)
cdef pieces_onehot(int[:,:] board_np_view, bint to_play):
    cdef int N = 8
    cdef int P = 6 # Number of pieces piece
    cdef np.ndarray onehot_w = np.zeros([N, N, P], dtype=DTYPE)
    cdef int[:, :, :] onehot_w_view = onehot_w
    cdef np.ndarray onehot_b = np.zeros([N, N, P], dtype=DTYPE)
    cdef int[:, :, :] onehot_b_view = onehot_b

    cdef Py_ssize_t i, j, p
    cdef Py_ssize_t _P = 6

    for i in range(N):
        for j in range(N):
            for p in range(_P):
                if board_np_view[i, j] == p + 1:
                    onehot_w_view[i, j, p] = 1
                elif board_np_view[i, j] == -(p + 1):
                    onehot_b_view[i, j, p] = 1

    if to_play: # If player one is white
        return onehot_w, onehot_b
    else: # If player one is black -> flip perspective
        return np.flip(np.flip(onehot_b, axis=0), axis=1), np.flip(np.flip(onehot_w, axis=0), axis=1)

@cython.boundscheck(False)
@cython.wraparound(False)
def board_to_image_c(board: chess.Board):
    cdef int N = 8
    cdef int T = 8
    cdef int M = 14
    cdef int L = 8
    cdef int P = 8
    cdef bint current_player = board.turn
    cdef np.ndarray image = np.zeros([N, N, T * M + L], dtype=DTYPE)
    cdef int t
    cdef np.ndarray[DTYPE_t, ndim=2] board_np
    cdef np.ndarray[DTYPE_t, ndim=3] p1
    cdef np.ndarray[DTYPE_t, ndim=3] p2
    cdef Py_ssize_t _T = 8
    cdef Py_ssize_t idx

    tmp = board.copy()
    for t in range(_T):
        idx = (T - t - 1) * M
        #board_np = parse_epd(tmp.epd())
        board_np = parse_piece_map(tmp.piece_map())
        p1, p2 = pieces_onehot(board_np, current_player)
        image[:, :, idx:idx+6] = p1
        image[:, :, idx + 6:idx+12] = p2
        if tmp.is_repetition(2):
            image[:, :, idx + 12] = 1
            if tmp.is_repetition(3):
                image[:, :, idx + 13] = 1
        if len(tmp.move_stack) > 0:
            tmp.pop()
        else:
            break
    image[:, :, 112] = int(current_player) # 1 if white 0 if black
    image[:, :, 113] = len(board.move_stack)
    image[:, :, 114] = board.halfmove_clock
    image[:, :, 115] = int(board.has_kingside_castling_rights(current_player))
    image[:, :, 116] = int(board.has_queenside_castling_rights(current_player))
    image[:, :, 117] = int(board.has_kingside_castling_rights(not current_player))
    image[:, :, 118] = int(board.has_queenside_castling_rights(not current_player))
    return image

@cython.boundscheck(False)
@cython.wraparound(False)
cdef copy_history(np.ndarray[DTYPE_t, ndim=3] new_image, np.ndarray[DTYPE_t, ndim=3] prev_image):
    cdef np.ndarray[DTYPE_t, ndim=3] tmp
    cdef Py_ssize_t T = 7
    cdef Py_ssize_t t
    cdef Py_ssize_t _from1, _from2, _to1, _to2
    cdef int M = 14

    tmp = np.flip(prev_image, axis=0)
    tmp = np.flip(tmp, axis=1)
    tmp = np.roll(tmp, shift=-M)
    for t in range(T):
        _from1 = (t * M)
        _from2 = (t * M) + 6
        _to1 = (t * M) + 6
        _to2 = (t * M) + 12
        new_image[:, :, _from1:_to1] = tmp[:, :, _from2:_to2]
        new_image[:, :, _from2:_to2] = tmp[:, :, _from1:_to1]

    return new_image

@cython.boundscheck(False)
@cython.wraparound(False)
def update_image(board, prev_image):
    cdef int N = 8
    cdef int T = 8
    cdef int M = 14
    cdef int L = 7
    cdef bint current_player = board.turn
    cdef np.ndarray[DTYPE_t, ndim=3] p1
    cdef np.ndarray[DTYPE_t, ndim=3] p2
    cdef np.ndarray new_image = np.zeros((N, N, (T*M)+L), dtype=DTYPE)
    cdef np.ndarray board_np = parse_epd(board.epd())

    new_image = copy_history(new_image, prev_image)

    # fill in missing info of current timestep
    p1, p2 = pieces_onehot(board_np, current_player)
    new_image[:, :, 98:104] = p1
    new_image[:, :, 104:110] = p2
    if board.is_repetition(2):
        new_image[:, :, 110] = 1
        if board.is_repetition(3):
            new_image[:, :, 111] = 1
    new_image[:, :, 112] = int(current_player) # 1 if white 0 if black
    new_image[:, :, 113] = len(board.move_stack)
    new_image[:, :, 114] = board.halfmove_clock
    new_image[:, :, 115] = int(board.has_kingside_castling_rights(current_player))
    new_image[:, :, 116] = int(board.has_queenside_castling_rights(current_player))
    new_image[:, :, 117] = int(board.has_kingside_castling_rights(not current_player))
    new_image[:, :, 118] = int(board.has_queenside_castling_rights(not current_player))
    
    return new_image

def test():
    board = chess.Board()
    board.push_san("e4")
    board.push_san("e5")
    board.push_san("a3")
    board.push_san("d5")
    board.push_san("Nc3")
    board.push_san("Nc6")
    #image = board_to_image_c(board)
    for i in range(10001):
        image = board_to_image_c(board)
        #update_image(board, image)

cProfile.runctx("test()", globals(), locals(), "Profile.prof")

s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

Content of stderr:
In file included from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /home/tomaz/.cache/ipython/cython/_cython_magic_48152ce4b85df3bc78bae332af01974c234c0e76.c:1265:
/home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~Tue Feb 13 10:21:37 2024    Profile.prof

         20702414 function calls in 6.760 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  2240224    

In [110]:
import numpy as np
import chess

board = chess.Board()
pieces = {
            "p": -1, "n": -2, "b": -3, "r": -4, "q": -5, "k": -6,
            "P": 1, "N": 2, "B": 3, "R": 4, "Q": 5, "K": 6
        }
pieces_map = board.piece_map()
board_np = np.zeros((8, 8), dtype=int)
for idx, piece in pieces_map.items():
    row = int(idx) // 8
    col = int(idx) % 8
    board_np[row, col] = pieces[piece.symbol()]


{63: Piece.from_symbol('r'), 62: Piece.from_symbol('n'), 61: Piece.from_symbol('b'), 60: Piece.from_symbol('k'), 59: Piece.from_symbol('q'), 58: Piece.from_symbol('b'), 57: Piece.from_symbol('n'), 56: Piece.from_symbol('r'), 55: Piece.from_symbol('p'), 54: Piece.from_symbol('p'), 53: Piece.from_symbol('p'), 52: Piece.from_symbol('p'), 51: Piece.from_symbol('p'), 50: Piece.from_symbol('p'), 49: Piece.from_symbol('p'), 48: Piece.from_symbol('p'), 15: Piece.from_symbol('P'), 14: Piece.from_symbol('P'), 13: Piece.from_symbol('P'), 12: Piece.from_symbol('P'), 11: Piece.from_symbol('P'), 10: Piece.from_symbol('P'), 9: Piece.from_symbol('P'), 8: Piece.from_symbol('P'), 7: Piece.from_symbol('R'), 6: Piece.from_symbol('N'), 5: Piece.from_symbol('B'), 4: Piece.from_symbol('K'), 3: Piece.from_symbol('Q'), 2: Piece.from_symbol('B'), 1: Piece.from_symbol('N'), 0: Piece.from_symbol('R')}


In [111]:
board_np

array([[ 4,  2,  3,  5,  6,  3,  2,  4],
       [ 1,  1,  1,  1,  1,  1,  1,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-4, -2, -3, -5, -6, -3, -2, -4]])

In [82]:
neki = pieces_map[5]

In [83]:
neki.symbol()

'B'

In [84]:
idx = 6

idx % 8

6

In [85]:
idx - 8 * (idx % 8)

-42

In [106]:
z = np.zeros((8, 8), dtype=int)
for i in range(64):
    row = i // 8
    col = i % 8
    z[row, col] = 1
    print(f"row {row}, col {col}")
print(z)

row 0, col 0
row 0, col 1
row 0, col 2
row 0, col 3
row 0, col 4
row 0, col 5
row 0, col 6
row 0, col 7
row 1, col 0
row 1, col 1
row 1, col 2
row 1, col 3
row 1, col 4
row 1, col 5
row 1, col 6
row 1, col 7
row 2, col 0
row 2, col 1
row 2, col 2
row 2, col 3
row 2, col 4
row 2, col 5
row 2, col 6
row 2, col 7
row 3, col 0
row 3, col 1
row 3, col 2
row 3, col 3
row 3, col 4
row 3, col 5
row 3, col 6
row 3, col 7
row 4, col 0
row 4, col 1
row 4, col 2
row 4, col 3
row 4, col 4
row 4, col 5
row 4, col 6
row 4, col 7
row 5, col 0
row 5, col 1
row 5, col 2
row 5, col 3
row 5, col 4
row 5, col 5
row 5, col 6
row 5, col 7
row 6, col 0
row 6, col 1
row 6, col 2
row 6, col 3
row 6, col 4
row 6, col 5
row 6, col 6
row 6, col 7
row 7, col 0
row 7, col 1
row 7, col 2
row 7, col 3
row 7, col 4
row 7, col 5
row 7, col 6
row 7, col 7
[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]
